<a href="https://colab.research.google.com/github/a-forty-two/cylons/blob/master/09_Word_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [3]:
# BODY -> body of string from which we will dervice our vectors
corpus_raw = 'Richie Rich is rich. The rich person is a happy person. He and She are not rich.'
corpus_lower = corpus_raw.lower()
corpus_lower

'richie rich is rich. the rich person is a happy person. he and she are not rich.'

In [5]:
words = []
for word in corpus_lower.split():
  if word != '.': # independent punctuation (not directly next to a word)
    # if punctuation is not individual word but a part of word-> bye.
    word = word.replace('.', '') # every punctuation that we don't want should be filtered here
    words.append(word)

print(words)
# TO CONVERT TO VECTORS , we don't want repeating! 
# Which DS allows us to avoid repeating? SET!!
print(len(words))
unique_words = set(words)
print(unique_words)
print(len(unique_words))

['richie', 'rich', 'is', 'rich', 'the', 'rich', 'person', 'is', 'a', 'happy', 'person', 'he', 'and', 'she', 'are', 'not', 'rich']
17
{'a', 'not', 'is', 'she', 'richie', 'he', 'are', 'and', 'the', 'rich', 'person', 'happy'}
12


In [7]:
word2int = {} # this was wht we had in imdb yesterday (word_index)
int2word = {}
vocab_size = len(unique_words)
for i,word in enumerate(unique_words):
  word2int[word] =i 
  int2word[i] = word

print(word2int)
print(int2word)
print(word2int['rich'])

{'a': 0, 'not': 1, 'is': 2, 'she': 3, 'richie': 4, 'he': 5, 'are': 6, 'and': 7, 'the': 8, 'rich': 9, 'person': 10, 'happy': 11}
{0: 'a', 1: 'not', 2: 'is', 3: 'she', 4: 'richie', 5: 'he', 6: 'are', 7: 'and', 8: 'the', 9: 'rich', 10: 'person', 11: 'happy'}
9


In [10]:
# SENTENCE level tokens so that we know what a complete sample looks like!
# please note: i have only 3 sentences, so dicitonary, prediction everything is going
# to be very poor!

raw_sentences = corpus_raw.split('.') # now breaking sentences instead of ' '
sentences = []
for sentence in raw_sentences:
  sentences.append(sentence.split()) # SPLIT -> BOTH SENTENCE and WORD tokens are created in 1go!

print(sentences) # contains extra [] because of the LAST '.' 
# sentences split the last full stop also became and element! 
sentences = sentences[:-1]
sentences

[['Richie', 'Rich', 'is', 'rich'], ['The', 'rich', 'person', 'is', 'a', 'happy', 'person'], ['He', 'and', 'She', 'are', 'not', 'rich'], []]


[['Richie', 'Rich', 'is', 'rich'],
 ['The', 'rich', 'person', 'is', 'a', 'happy', 'person'],
 ['He', 'and', 'She', 'are', 'not', 'rich']]

In [12]:
# i wanted to create word prediction
# ML algo?
# y-> next word, x -> previous word
# y = weights * x + bias 
# next_word = weights * previous_word + bias 

# BASICALLY what i need is ALL possible pairs of WORDs that can follow each other!!!

window_size = 2 # 2 words at a time
data = [] 
for sentence in sentences: # loop in sentence
  for wordId, word in enumerate(sentence): # loop in word
    for nxword in sentence[max(wordId-window_size,0):min(wordId+window_size,len(sentence))+1]:  
      if nxword!=word:
        data.append([word,nxword])
         # every word that could come next to it! 
      # no. of elements = len-window_size 
      # max -> 0, 1 at that point 0-2 and 1-2 will give me neg nos. hence error
      # to fix it we used max with 0! 
      # min -> at the end of array-> to avoid crossing the max len of array! 
      # +1 because the last element is excluded in numpy ranges [1,2]-> from 1 till 1!

print(data)


[['Richie', 'Rich'], ['Richie', 'is'], ['Rich', 'Richie'], ['Rich', 'is'], ['Rich', 'rich'], ['is', 'Richie'], ['is', 'Rich'], ['is', 'rich'], ['rich', 'Rich'], ['rich', 'is'], ['The', 'rich'], ['The', 'person'], ['rich', 'The'], ['rich', 'person'], ['rich', 'is'], ['person', 'The'], ['person', 'rich'], ['person', 'is'], ['person', 'a'], ['is', 'rich'], ['is', 'person'], ['is', 'a'], ['is', 'happy'], ['a', 'person'], ['a', 'is'], ['a', 'happy'], ['a', 'person'], ['happy', 'is'], ['happy', 'a'], ['happy', 'person'], ['person', 'a'], ['person', 'happy'], ['He', 'and'], ['He', 'She'], ['and', 'He'], ['and', 'She'], ['and', 'are'], ['She', 'He'], ['She', 'and'], ['She', 'are'], ['She', 'not'], ['are', 'and'], ['are', 'She'], ['are', 'not'], ['are', 'rich'], ['not', 'She'], ['not', 'are'], ['not', 'rich'], ['rich', 'are'], ['rich', 'not']]
